In [2]:
import os
import numpy as np
import h5py

from torchvision import transforms

from src.era5_dataset import ERA5Dataset,means,stds

# ee.Authenticate()
# ee.Initialize(project='eofm-benchmark')

In [ ]:
california_lat_min = 32.3
california_lat_max = 42.2

california_lon_min = -124.7
california_lon_max = -113.9

path = 'datasets/data_era5_2016_02_data_stream-oper_stepType-instant.nc'

climate_vars = ['t2m','u10','v10']
years = ['2020']

months  = [
    str(i).rjust(2,'0') for i in range(1,13,1)
]


In [ ]:
means = np.zeros(len(climate_vars))
stds = np.zeros(len(climate_vars))

s = np.zeros(len(climate_vars))
sum_of_squares = np.zeros(len(climate_vars))

count = 0

for month in months:
    path = f'datasets/{years[0]}/{month}/data_stream-oper_stepType-instant.nc'

    with h5py.File(path) as f:
        count += len(f['expver']) * len(f['latitude']) * len(f['longitude'])
        for i,var in enumerate(climate_vars):
            s[i] += np.sum(f[var])
            sum_of_squares[i] += np.sum(np.square(f[var]))

means = s / count
stds = np.sqrt((sum_of_squares / count) - np.square(s / count))

In [ ]:
print(f'{means}')
print(f'{stds}')

In [19]:
path = f'datasets/era5_new/{years[0]}/{months[0]}/data_stream-oper_stepType-instant.nc'

with h5py.File(path,'r') as f:
    # f.create_group('01')
    print(f.keys())
    print(f['expver'].shape)
    print(f['expver'].shape[0] // 24)

<KeysViewHDF5 ['expver', 'latitude', 'longitude', 'number', 'sp', 't2m', 'tcw', 'u10', 'v10', 'valid_time']>
(744,)
31


In [ ]:
# years = ['2016','2017','2018','2019','2020','2021','2022']
# months  = [
#     str(i).rjust(2,'0') for i in range(1,13,1)
# ]

years = ['2020','2021']
months  = [
    str(i).rjust(2,'0') for i in range(1,13,1)
]


weather_vars = ['sp', 't2m', 'tcw', 'u10', 'v10']

for year in years:
    for month in months:
        orig_path = f'datasets/era5_new/{year}/{month}/data_stream-oper_stepType-instant.nc'
        path = f'datasets/era5_new/{year}/{month}/hourly_data.h5'

        file = h5py.File(orig_path,mode='r')
        hrs = file['expver']
        lat = file['latitude']
        lon = file['longitude']

        new_file = h5py.File(path,mode='w')

        new_file.create_dataset('hrs',data=hrs)
        new_file.create_dataset('latitude',data=lat)
        new_file.create_dataset('longitude',data=lon)
        new_file.create_group('hourly_data')


        for i in range(hrs.shape[0]):
            new_file.create_group(f'hourly_data/{i}')
        for clim_var in weather_vars:
            arr = file[clim_var]
            for j in range(hrs.shape[0]):
                data_point = arr[j,:,:]
                new_file.create_dataset(f'hourly_data/{j}/{clim_var}',data=data_point)


In [11]:
with h5py.File('./datasets/era5_new/2022/01/hourly_data.h5') as f:
    print(f.keys())

    print(f['hourly_data']['234']['t2m'].shape)

<KeysViewHDF5 ['hourly_data', 'hrs', 'latitude', 'longitude']>
(721, 1440)


In [ ]:
import numpy as np
from torchvision import transforms
from src.era5_dataset import ERA5Dataset,means,stds
from torch.utils.data import DataLoader, Subset

from timeit import timeit

args = {
    'data_path':'/home/rpdemilt/climate_downscaling/neural-operators-weather-downscaling/datasets/era5_new/',
}

climate_vars = [
    't2m',
    'u10',
    'v10'
]

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[means[clim_var] for clim_var in climate_vars],
            std=[stds[clim_var] for clim_var in climate_vars]
        )
    ])

dataset = ERA5Dataset(
            location=args['data_path'],
            train=False,
            transform=transform,
            climate_vars=climate_vars
        )

n = 2000

subset = Subset(
    dataset=dataset,
    indices = np.arange(len(dataset))[:n]
)

dataloader = DataLoader(
    dataset=subset,
    batch_size=32,
    num_workers=8
)

def time_access(dataset):
    for i, batch in enumerate(dataset):
        X,y = batch
